##  Part VIII: Lazy Predict. | Jupyter Notebook

## Final Project Submission

Please fill out:
* __Student name:__ Sharonda Pettiett-Warner
* __Student pace:__ part time - PT_0610
* __Scheduled project review date/time:__ April 24, 2020
* __Instructor name:__ Eli Thomas
* __Blog post URL:__ TBD

# Begin Study
> In this notebook I will use the 'Lazy Predict' algorithm to build several baseline models with basic feature engineering. This study will examine the hotel booking dataset from a kernel in Kaggle. The objective is to build a lot of basic models without much code which helps to understand which models work better without any parameter tuning.

# Import Libraries

In [2]:
import numpy as np 
import pandas as pd 
import sys
import os 

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier


from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import auc, classification_report,  confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
#from sklearn.preprocessing import Imputer

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="whitegrid")

import warnings
warnings.filterwarnings('ignore')

In [12]:
import datetime
start = datetime.datetime.now()

In [ ]:
pd.set_option('display.max_columns', None)

# Load Dataset

In [3]:
dataset = pd.read_csv('./hotel_bookings.csv')
dataset.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [4]:
dataset.shape

(119390, 32)

# EDA

In [5]:
dataset.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')

# Lazy Predict 
> Lazy Predict help build a lot of basic models without much code and helps understand which models works better without any parameter tuning.
- Free software: MIT license | __Documentation:__ https://lazypredict.readthedocs.io.

#### Documentation

> URL: https://lazypredict.readthedocs.io/en/latest/installation.html

#### Author
> Shankar Rao Pandala

#### pip install

In [6]:
#!pip install lazypredict
#!pip install catboost
#!pip install xgboost
#!pip install lightgbm
#!pip install --upgrade scikit-learn

#### Import Library

In [7]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.svm import SVC
enable_hist_gradient_boosting
import scipy
from sklearn.utils import deprecated
from sklearn.utils._fast_dict import IntFloatDict
from sklearn.utils.fixes import _astype_copy_false

In [8]:
import lazypredict
from lazypredict.Supervised import LazyClassifier   #LazyRegressor
from sklearn.model_selection import train_test_split

import sys

### Classification Example - Hotel Booking Cancellations

#### Drop Columns

In [9]:
y= dataset['is_canceled']
X = dataset.drop(['is_canceled', 'company', 'booking_changes', 'reservation_status','days_in_waiting_list'], axis=1)
display(y.head())
display(X.head())

0    0
1    0
2    0
3    0
4    0
Name: is_canceled, dtype: int64

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,deposit_type,agent,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date
0,Resort Hotel,342,2015,July,27,1,0,0,2,0.00,...,0,C,C,No Deposit,nan,Transient,0.00,0,0,2015-07-01
1,Resort Hotel,737,2015,July,27,1,0,0,2,0.00,...,0,C,C,No Deposit,nan,Transient,0.00,0,0,2015-07-01
2,Resort Hotel,7,2015,July,27,1,0,1,1,0.00,...,0,A,C,No Deposit,nan,Transient,75.00,0,0,2015-07-02
3,Resort Hotel,13,2015,July,27,1,0,1,1,0.00,...,0,A,A,No Deposit,304.00,Transient,75.00,0,0,2015-07-02
4,Resort Hotel,14,2015,July,27,1,0,2,2,0.00,...,0,A,A,No Deposit,240.00,Transient,98.00,0,1,2015-07-03


#### Split train_test data

In [ ]:
# Create train_test split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)

#### Run Lazy Predict - LazyClassifier()

In [10]:
# Execute LazyClassifier()
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None, predictions=True) 
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

 97%|████████████████████████████████████████████████████████████████████████████▍  | 30/31 [6:34:19<15:46, 946.25s/it]

Learning rate set to 0.053298
0:	learn: 0.6421835	total: 144ms	remaining: 2m 24s
1:	learn: 0.6035997	total: 182ms	remaining: 1m 30s
2:	learn: 0.5718637	total: 224ms	remaining: 1m 14s
3:	learn: 0.5419864	total: 262ms	remaining: 1m 5s
4:	learn: 0.5205076	total: 299ms	remaining: 59.4s
5:	learn: 0.5018892	total: 335ms	remaining: 55.5s
6:	learn: 0.4837301	total: 372ms	remaining: 52.8s
7:	learn: 0.4685411	total: 409ms	remaining: 50.7s
8:	learn: 0.4551399	total: 446ms	remaining: 49.1s
9:	learn: 0.4454744	total: 484ms	remaining: 47.9s
10:	learn: 0.4368193	total: 521ms	remaining: 46.9s
11:	learn: 0.4302282	total: 556ms	remaining: 45.8s
12:	learn: 0.4243306	total: 594ms	remaining: 45.1s
13:	learn: 0.4187576	total: 630ms	remaining: 44.4s
14:	learn: 0.4135999	total: 664ms	remaining: 43.6s
15:	learn: 0.4064655	total: 701ms	remaining: 43.1s
16:	learn: 0.4016449	total: 739ms	remaining: 42.7s
17:	learn: 0.3964782	total: 775ms	remaining: 42.3s
18:	learn: 0.3916703	total: 838ms	remaining: 43.3s
19:	lear

160:	learn: 0.3032333	total: 5.91s	remaining: 30.8s
161:	learn: 0.3030406	total: 5.94s	remaining: 30.7s
162:	learn: 0.3029315	total: 5.97s	remaining: 30.7s
163:	learn: 0.3026514	total: 6.01s	remaining: 30.6s
164:	learn: 0.3024891	total: 6.04s	remaining: 30.6s
165:	learn: 0.3024232	total: 6.08s	remaining: 30.5s
166:	learn: 0.3021941	total: 6.11s	remaining: 30.5s
167:	learn: 0.3020471	total: 6.14s	remaining: 30.4s
168:	learn: 0.3019605	total: 6.18s	remaining: 30.4s
169:	learn: 0.3012289	total: 6.21s	remaining: 30.3s
170:	learn: 0.3010083	total: 6.25s	remaining: 30.3s
171:	learn: 0.3007802	total: 6.28s	remaining: 30.3s
172:	learn: 0.3005458	total: 6.32s	remaining: 30.2s
173:	learn: 0.3004478	total: 6.35s	remaining: 30.2s
174:	learn: 0.3003285	total: 6.39s	remaining: 30.1s
175:	learn: 0.3001244	total: 6.42s	remaining: 30.1s
176:	learn: 0.2999776	total: 6.46s	remaining: 30s
177:	learn: 0.2997369	total: 6.5s	remaining: 30s
178:	learn: 0.2992290	total: 6.53s	remaining: 30s
179:	learn: 0.29899

319:	learn: 0.2714968	total: 11.7s	remaining: 24.9s
320:	learn: 0.2712970	total: 11.7s	remaining: 24.8s
321:	learn: 0.2711667	total: 11.8s	remaining: 24.8s
322:	learn: 0.2709722	total: 11.8s	remaining: 24.7s
323:	learn: 0.2707781	total: 11.8s	remaining: 24.7s
324:	learn: 0.2705738	total: 11.9s	remaining: 24.7s
325:	learn: 0.2703976	total: 11.9s	remaining: 24.6s
326:	learn: 0.2702548	total: 12s	remaining: 24.6s
327:	learn: 0.2700770	total: 12s	remaining: 24.6s
328:	learn: 0.2699482	total: 12.1s	remaining: 24.6s
329:	learn: 0.2698083	total: 12.1s	remaining: 24.6s
330:	learn: 0.2695931	total: 12.2s	remaining: 24.6s
331:	learn: 0.2694035	total: 12.2s	remaining: 24.6s
332:	learn: 0.2693060	total: 12.3s	remaining: 24.6s
333:	learn: 0.2691942	total: 12.3s	remaining: 24.6s
334:	learn: 0.2690477	total: 12.4s	remaining: 24.5s
335:	learn: 0.2688083	total: 12.4s	remaining: 24.5s
336:	learn: 0.2686456	total: 12.5s	remaining: 24.5s
337:	learn: 0.2685396	total: 12.5s	remaining: 24.5s
338:	learn: 0.26

478:	learn: 0.2459074	total: 18s	remaining: 19.6s
479:	learn: 0.2457414	total: 18s	remaining: 19.6s
480:	learn: 0.2455955	total: 18.1s	remaining: 19.5s
481:	learn: 0.2454285	total: 18.1s	remaining: 19.5s
482:	learn: 0.2453024	total: 18.2s	remaining: 19.4s
483:	learn: 0.2452266	total: 18.2s	remaining: 19.4s
484:	learn: 0.2450468	total: 18.2s	remaining: 19.4s
485:	learn: 0.2449687	total: 18.3s	remaining: 19.3s
486:	learn: 0.2449352	total: 18.3s	remaining: 19.3s
487:	learn: 0.2447934	total: 18.4s	remaining: 19.3s
488:	learn: 0.2446502	total: 18.4s	remaining: 19.2s
489:	learn: 0.2445009	total: 18.4s	remaining: 19.2s
490:	learn: 0.2443559	total: 18.5s	remaining: 19.1s
491:	learn: 0.2441571	total: 18.5s	remaining: 19.1s
492:	learn: 0.2440092	total: 18.6s	remaining: 19.1s
493:	learn: 0.2439062	total: 18.6s	remaining: 19s
494:	learn: 0.2437819	total: 18.6s	remaining: 19s
495:	learn: 0.2435907	total: 18.7s	remaining: 19s
496:	learn: 0.2434241	total: 18.7s	remaining: 18.9s
497:	learn: 0.2432462	

637:	learn: 0.2244052	total: 23.5s	remaining: 13.4s
638:	learn: 0.2243141	total: 23.6s	remaining: 13.3s
639:	learn: 0.2241750	total: 23.6s	remaining: 13.3s
640:	learn: 0.2241468	total: 23.6s	remaining: 13.2s
641:	learn: 0.2240536	total: 23.7s	remaining: 13.2s
642:	learn: 0.2239671	total: 23.7s	remaining: 13.2s
643:	learn: 0.2238905	total: 23.7s	remaining: 13.1s
644:	learn: 0.2238190	total: 23.8s	remaining: 13.1s
645:	learn: 0.2236775	total: 23.8s	remaining: 13s
646:	learn: 0.2235824	total: 23.8s	remaining: 13s
647:	learn: 0.2235722	total: 23.9s	remaining: 13s
648:	learn: 0.2234442	total: 23.9s	remaining: 12.9s
649:	learn: 0.2233178	total: 23.9s	remaining: 12.9s
650:	learn: 0.2231808	total: 24s	remaining: 12.9s
651:	learn: 0.2230362	total: 24s	remaining: 12.8s
652:	learn: 0.2229019	total: 24s	remaining: 12.8s
653:	learn: 0.2227365	total: 24.1s	remaining: 12.7s
654:	learn: 0.2226436	total: 24.1s	remaining: 12.7s
655:	learn: 0.2226259	total: 24.1s	remaining: 12.7s
656:	learn: 0.2225310	to

796:	learn: 0.2066668	total: 29s	remaining: 7.39s
797:	learn: 0.2066044	total: 29.1s	remaining: 7.36s
798:	learn: 0.2065380	total: 29.1s	remaining: 7.32s
799:	learn: 0.2064291	total: 29.1s	remaining: 7.28s
800:	learn: 0.2062843	total: 29.2s	remaining: 7.25s
801:	learn: 0.2061781	total: 29.2s	remaining: 7.21s
802:	learn: 0.2060867	total: 29.2s	remaining: 7.17s
803:	learn: 0.2060090	total: 29.3s	remaining: 7.13s
804:	learn: 0.2058873	total: 29.3s	remaining: 7.1s
805:	learn: 0.2057875	total: 29.3s	remaining: 7.06s
806:	learn: 0.2056814	total: 29.4s	remaining: 7.02s
807:	learn: 0.2055233	total: 29.4s	remaining: 6.99s
808:	learn: 0.2054252	total: 29.4s	remaining: 6.95s
809:	learn: 0.2052405	total: 29.5s	remaining: 6.91s
810:	learn: 0.2051500	total: 29.5s	remaining: 6.88s
811:	learn: 0.2050733	total: 29.5s	remaining: 6.84s
812:	learn: 0.2050220	total: 29.6s	remaining: 6.8s
813:	learn: 0.2049586	total: 29.6s	remaining: 6.76s
814:	learn: 0.2048505	total: 29.6s	remaining: 6.73s
815:	learn: 0.20

955:	learn: 0.1909179	total: 34.4s	remaining: 1.58s
956:	learn: 0.1908221	total: 34.5s	remaining: 1.55s
957:	learn: 0.1907358	total: 34.5s	remaining: 1.51s
958:	learn: 0.1906245	total: 34.5s	remaining: 1.48s
959:	learn: 0.1905288	total: 34.6s	remaining: 1.44s
960:	learn: 0.1904248	total: 34.6s	remaining: 1.41s
961:	learn: 0.1903444	total: 34.7s	remaining: 1.37s
962:	learn: 0.1902771	total: 34.7s	remaining: 1.33s
963:	learn: 0.1901747	total: 34.7s	remaining: 1.3s
964:	learn: 0.1900976	total: 34.8s	remaining: 1.26s
965:	learn: 0.1899835	total: 34.8s	remaining: 1.23s
966:	learn: 0.1898753	total: 34.8s	remaining: 1.19s
967:	learn: 0.1897853	total: 34.9s	remaining: 1.15s
968:	learn: 0.1896884	total: 34.9s	remaining: 1.12s
969:	learn: 0.1895722	total: 34.9s	remaining: 1.08s
970:	learn: 0.1894939	total: 35s	remaining: 1.04s
971:	learn: 0.1893904	total: 35s	remaining: 1.01s
972:	learn: 0.1893109	total: 35s	remaining: 972ms
973:	learn: 0.1892247	total: 35.1s	remaining: 936ms
974:	learn: 0.18914

100%|███████████████████████████████████████████████████████████████████████████████| 31/31 [6:35:31<00:00, 684.01s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
QuadraticDiscriminantAnalysis,0.95,0.93,0.93,0.95,35.37
CatBoostClassifier,0.93,0.92,0.92,0.93,72.11
ExtraTreesClassifier,0.90,0.88,0.88,0.90,158.01
BaggingClassifier,0.90,0.88,0.88,0.90,122.54
RandomForestClassifier,0.90,0.88,0.88,0.90,99.79
CalibratedClassifierCV,0.90,0.88,0.88,0.90,69.34
LinearSVC,0.90,0.88,0.88,0.90,21.72
DecisionTreeClassifier,0.88,0.87,0.87,0.88,25.49
LGBMClassifier,0.88,0.87,0.87,0.88,8.59


In [14]:
end = datetime.datetime.now()
elapsed_time = end - start
print(f'Elapsed Time: {elapsed_time}')